In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_not_found.csv')

print(f"행 개수: {len(df)}")

행 개수: 665


In [6]:
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings_only.csv')

print(f"행 개수: {len(df)}")

행 개수: 2179


In [7]:
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/ffinal_pneumo_sampled_cleaned.csv')

print(f"행 개수: {len(df)}")

행 개수: 2440


In [12]:
import pandas as pd

# CSV 파일 읽기
df1 = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings_only.csv")
df2 = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/ffinal_pneumo_sampled_cleaned.csv")

rows1 = set([tuple(row) for row in df1.astype(str).to_numpy()])
rows2 = set([tuple(row) for row in df2.astype(str).to_numpy()])

added_rows = rows2 - rows1  # file2에만 있는 행
removed_rows = rows1 - rows2  # file1에만 있는 행

print("추가된 행:", added_rows)
print("삭제된 행:", removed_rows)



추가된 행: {('25419b07-1df26981-761bdd0e-5bc109d2-237aa761.jpg', 'There are bibasilar opacities, with a correlate projecting over the spine on\n the lateral view, findings concerning for pneumonia.  Upper lungs are clear. \n Specifically, previously noted right upper lobe opacity in ___ has\n resolved.  No wall pleural effusion or pneumothorax.  Mild cardiomegaly. \n Mediastinal contours are normal.  No subdiaphragmatic free air.', 'pneumonia'), ('8ee88a48-220d7538-c746eff2-26e219e6-b743d2a4.jpg', 'The left mid to upper lobe opacification is new from prior and could represent\n a focus of infection. The bilateral hemidiaphragms, cardiac borders, and\n mediastinal silhouettes are normal without pleural effusion or pneumothorax.', 'pneumonia'), ('8d50760b-73747607-21ed8e1a-018ed84c-17269527.jpg', 'AP and lateral views of the chest were obtained.  Patchy left base\n patchy is worrisome for infection.  There is additional focus of rounded\n opacity projecting over the posterior left seventh ri

In [13]:
import os
folder_path = "/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_study"  # 폴더 경로 지정

# 폴더 안의 파일 개수 세기 (하위 폴더 제외)
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"파일 개수: {file_count}")

파일 개수: 2844


In [18]:
import pandas as pd

# 입력 파일 경로
csv1 = "/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings_only.csv"   # studyID, FINDINGS 가 있는 파일
csv2 = "/Users/skku_aws19/Desktop/aws_project/final_project/matched_mimic_files.csv"  # study_id, file_name 이 있는 파일
out  = "/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings.csv"

# 1) 로드
df1 = pd.read_csv(csv1)
df2 = pd.read_csv(csv2)

# 2) 키 정규화 (문자열로 변환 + 공백 제거)
def norm_key(s):
    return s.astype(str).str.strip()

if "studyID" not in df1.columns:
    raise ValueError("first.csv에 'studyID' 컬럼이 없습니다.")
if "FINDINGS" not in df1.columns:
    raise ValueError("first.csv에 'FINDINGS' 컬럼이 없습니다.")
if "study_id" not in df2.columns:
    raise ValueError("second.csv에 'study_id' 컬럼이 없습니다.")
if "file_name" not in df2.columns:
    raise ValueError("second.csv에 'file_name' 컬럼이 없습니다.")

df1["__key"] = norm_key(df1["studyID"])
df2["__key"] = norm_key(df2["study_id"])

# (선택) 키 중복 처리: 중복이 있다면 첫 행만 사용하거나, 원하는 규칙으로 정리
# 첫 번째 CSV: 같은 studyID가 여러 번 나오면 가장 첫 행만 사용
df1_dedup = df1.drop_duplicates(subset="__key", keep="first")
# 두 번째 CSV도 마찬가지
df2_dedup = df2.drop_duplicates(subset="__key", keep="first")

# 3) 내부조인(inner join)으로 겹치는 키만 결합
merged = pd.merge(
    df1_dedup[["__key", "FINDINGS"]],
    df2_dedup[["__key", "file_name"]],
    on="__key",
    how="inner"

)

# 4) 컬럼 정리 및 저장
merged = merged.rename(columns={"__key": "study_id"})[["study_id", "file_name", "FINDINGS"]]
merged.to_csv(out, index=False)

print(f"입력1 행수: {len(df1)} / 입력2 행수: {len(df2)}")
print(f"중복제거 후: first={len(df1_dedup)}, second={len(df2_dedup)}")
print(f"겹친 행수(출력): {len(merged)}")
print(f"저장 완료 → {out}")

입력1 행수: 2179 / 입력2 행수: 3180
중복제거 후: first=2179, second=2844
겹친 행수(출력): 2179
저장 완료 → /Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings.csv


In [16]:
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings.csv')

print(f"행 개수: {len(df)}")

행 개수: 2179


In [17]:
df = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings.csv")

# label 열 추가 (모든 값이 'pneumonia')
df["label"] = "pneumonia"

# 저장
df.to_csv("/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings_with_label.csv", index=False)

print("label 열이 추가된 CSV 저장 완료")

label 열이 추가된 CSV 저장 완료


In [ ]:
df = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/pneumo_findings_with_label.csv")

# file_name 뒤에 .jpg 붙이기 (이미 .jpg로 끝나는 건 중복 방지)
df["file_name"] = df["file_name"].astype(str)
df["file_name"] = df["file_name"].apply(lambda x: x if x.lower().endswith(".jpg") else f"{x}.jpg")

# 저장
df.to_csv("merged_file_name_findings_with_label_jpg.csv", index=False)

print(".jpg 확장자가 file_name 열에 추가됨")

In [1]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/ffinal_normal_sampled_dataset.csv")

# 1) 키워드가 없는 행 찾기
mask_no_keywords = ~(
    df["report"].str.contains("left", case=False, na=False) |
    df["report"].str.contains("right", case=False, na=False) |
    df["report"].str.contains("upper", case=False, na=False) |
    df["report"].str.contains("lower", case=False, na=False)
)

# 2) 키워드가 없는 행 중에서 261개만 선택해 삭제
rows_to_drop = df[mask_no_keywords].sample(n=261, random_state=42).index
df = df.drop(index=rows_to_drop)

# 3) 저장
df.to_csv("your_file_trimmed.csv", index=False)

print(f"최종 행 개수: {len(df)}")

최종 행 개수: 2179


In [3]:
import pandas as pd

# CSV 불러오기
df1 = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/data/final_normal.csv")
df2 = pd.read_csv("/Users/skku_aws19/Desktop/aws_project/final_project/data/final_pneumo.csv")

# 두 CSV 합치기 (행 방향)
df_merged = pd.concat([df1, df2], ignore_index=True)

# 저장
df_merged.to_csv("/Users/skku_aws19/Desktop/aws_project/final_project/data/merged.csv", index=False)

print(f"합친 후 행 개수: {len(df_merged)}")

합친 후 행 개수: 4358


In [3]:
import pandas as pd
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/data/test.csv')

print(f"행 개수: {len(df)}")

행 개수: 654


In [4]:
import pandas as pd
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/data/train.csv')

print(f"행 개수: {len(df)}")

행 개수: 3050


In [5]:

import pandas as pd
df = pd.read_csv('/Users/skku_aws19/Desktop/aws_project/final_project/data/val.csv')

print(f"행 개수: {len(df)}")

행 개수: 654


In [1]:
import os
import pandas as pd
import shutil

# CSV 경로
csv_path = "/Users/skku_aws19/Desktop/aws_project/final_project/data/merged.csv"

# report-dev 루트 폴더
report_dev_root = "/Users/skku_aws19/Desktop/aws_project/final_project/data/report-dev"

# 저장할 폴더
output_dir = "/Users/skku_aws19/Desktop/aws_project/final_project/data/imdat"
os.makedirs(output_dir, exist_ok=True)

# CSV 읽기 (file_name 컬럼이 있다고 가정)
df = pd.read_csv(csv_path)

# file_name 열에 .jpg 확실히 포함되어 있는지 체크
file_names = df["file_name"].astype(str).tolist()

# report-dev 하위 전체 탐색 후 매칭되는 파일 복사
count = 0
for root, _, files in os.walk(report_dev_root):
    for fname in files:
        if fname in file_names:
            src_path = os.path.join(root, fname)
            dst_path = os.path.join(output_dir, fname)
            shutil.copy2(src_path, dst_path)
            count += 1

print(f"총 {count}개의 파일을 {output_dir}에 저장했습니다.")


총 4358개의 파일을 /Users/skku_aws19/Desktop/aws_project/final_project/data/imdat에 저장했습니다.


In [7]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("data/merged_test_set.csv")

# file_name 값 앞에 data/imdat/ 붙이기
df["file_name"] = "data/imdat/" + df["file_name"].astype(str)

# 저장
df.to_csv("data/merged_test_set.csv", index=False)


In [2]:
import argparse
import pandas as pd
from pathlib import Path
import shutil
import sys



def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True, help="CSV path (must contain column 'file_name')")
    ap.add_argument("--src", required=True, help="Source root dir (preprocessed-old)")
    ap.add_argument("--dst", required=True, help="Destination dir to copy matched PNGs")
    ap.add_argument("--keep-tree", default="false", choices=["true","false"],
                    help="Preserve original subdirectory structure")
    args = ap.parse_args()

    csv_path = Path(args.csv)
    src = Path(args.src)
    dst = Path(args.dst)
    keep_tree = (args.keep_tree.lower() == "true")

    if not csv_path.exists():
        print(f"[ERR] CSV not found: {csv_path}", file=sys.stderr)
        sys.exit(1)
    if not src.exists():
        print(f"[ERR] Source dir not found: {src}", file=sys.stderr)
        sys.exit(1)
    dst.mkdir(parents=True, exist_ok=True)

    # 1) CSV에서 file_name 읽어와서 '확장자 제거한 스템' 세트 만들기
    df = pd.read_csv(csv_path)
    if "file_name" not in df.columns:
        print("[ERR] CSV must contain column 'file_name'", file=sys.stderr)
        sys.exit(1)

    stems = {
        Path(str(name).strip()).stem.lower()
        for name in df["file_name"].dropna().astype(str)
    }

    # 2) SRC 아래의 모든 .png 인덱스 만들기 (stem -> Path)
    index = {}
    duplicates = set()
    for p in src.rglob("*.png"):
        stem = p.stem.lower()
        if stem in index:
            duplicates.add(stem)  # 같은 stem의 PNG가 둘 이상 있는 경우
        else:
            index[stem] = p

    # 3) 매칭 복사
    copied = 0
    missing = []
    for stem in stems:
        src_path = index.get(stem)
        if src_path is None:
            missing.append(stem)
            continue

        if keep_tree:
            # 원본 구조 유지
            rel = src_path.relative_to(src)
            out_path = dst / rel
            out_path.parent.mkdir(parents=True, exist_ok=True)
        else:
            # 평평하게 모으기 (이름 충돌 방지)
            out_path = dst / src_path.name
            if out_path.exists():
                # 충돌 시 stem에 숫자 붙여 회피
                i = 1
                while True:
                    candidate = dst / f"{stem}_{i}.png"
                    if not candidate.exists():
                        out_path = candidate
                        break
                    i += 1

        shutil.copy2(src_path, out_path)
        copied += 1

    # 4) 로그 남기기
    (dst / "missing_from_csv.txt").write_text(
        "\n".join(missing) + ("\n" if missing else "")
    )
    (dst / "duplicate_stems_in_src.txt").write_text(
        "\n".join(sorted(duplicates)) + ("\n" if duplicates else "")
    )

    print(f"[DONE] Requested: {len(stems)} | Copied: {copied} | Missing: {len(missing)}")
    print(f" - Missing list: {dst / 'missing_from_csv.txt'}")
    print(f" - Duplicates   : {dst / 'duplicate_stems_in_src.txt'}")

if __name__ == "__main__":
    main()


/Library/Frameworks/Python.framework/Versions/3.12/Resources/Python.app/Contents/MacOS/Python: can't open file '/Users/skku_aws19/Desktop/aws_project/final_project/filter_copy_by_csv.py': [Errno 2] No such file or directory


usage: ipykernel_launcher.py [-h] --csv CSV --src SRC --dst DST
                             [--keep-tree {true,false}]
ipykernel_launcher.py: error: the following arguments are required: --csv, --src, --dst


SystemExit: 2

In [3]:
import pandas as pd

# 두 CSV 불러오기
train_df = pd.read_csv("data/train.csv")
val_df   = pd.read_csv("data/val.csv")

# 합치기 (행 방향으로)
merged_df = pd.concat([train_df, val_df], axis=0, ignore_index=True)

# 저장
merged_df.to_csv("train_val_merged.csv", index=False)

print("✅ train_val_merged.csv 저장 완료, 총 행 수:", len(merged_df))

✅ train_val_merged.csv 저장 완료, 총 행 수: 1700


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 합쳐둔 train+val 세트 불러오기
df = pd.read_csv("data/merged_trainval_set_clean.csv")

# 8:2 비율로 나누기 (random_state는 재현성 확보용)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# 저장
train_df.to_csv("data/train.csv", index=False)
val_df.to_csv("data/val.csv", index=False)

print("✅ 분할 완료")
print("Train size:", len(train_df))
print("Val size:", len(val_df))


✅ 분할 완료
Train size: 1600
Val size: 401


In [10]:
import os
import pandas as pd

# CSV 불러오기
trainval_path = "data/merged_trainval_set.csv"
test_path     = "data/test.csv"

df_trainval = pd.read_csv(trainval_path)
df_test     = pd.read_csv(test_path)

def drop_missing_files(df, filename_col="file_name"):
    exists_mask = df[filename_col].apply(lambda p: os.path.exists(p))
    print(f"[INFO] {len(df) - exists_mask.sum()} files not found, dropping...")
    return df[exists_mask].reset_index(drop=True)

# 경로에 없는 파일 삭제
df_trainval_clean = drop_missing_files(df_trainval)
df_test_clean     = drop_missing_files(df_test)

# 다시 저장
df_trainval_clean.to_csv("data/merged_trainval_set_clean.csv", index=False)
df_test_clean.to_csv("data/test_clean.csv", index=False)

print("✅ Cleaned CSV saved.")


[INFO] 178 files not found, dropping...
[INFO] 0 files not found, dropping...
✅ Cleaned CSV saved.
